In [1]:
import torch
print(torch.__version__)

2.8.0+cu126


In [2]:
!pip install -q nnunetv2 dynamic-network-architectures wandb nibabel SimpleITK

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 kB 6.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 7.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 4.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 MB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 99.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 109.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.4/96.4 kB 9.2 MB/s eta 0:00:00


In [35]:
import os, shutil, glob, csv, json
import nibabel as nib
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt

In [27]:
import wandb
wandb.login()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ryanwangk (ryanwangk-university-of-wisconsin-madison) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [7]:
DATASET_ID = 310
DATASET_NAME = f"Dataset{DATASET_ID}_Pancreas"

BASE_DIR = "/content/drive/MyDrive/PC-Classifier"
RAW_DIR = f"{BASE_DIR}/nnUNet_raw"
PREPROC_DIR = f"{BASE_DIR}/nnUNet_preprocessed"
RESULTS_DIR = f"{BASE_DIR}/nnUNet_results"

os.environ["nnUNet_raw"] = RAW_DIR
os.environ["nnUNet_preprocessed"] = PREPROC_DIR
os.environ["nnUNet_results"] = RESULTS_DIR
SRC_DIR = "/content/drive/MyDrive/ML-Quiz-3DMedImg"
DST_DIR = f"{RAW_DIR}/{DATASET_NAME}"
os.environ["PCCLS_LABELS"] = f"{DST_DIR}/subtype_labels.csv"

In [ ]:
!mkdir -p $RAW_DIR $PREPROC_DIR $RESULTS_DIR
!mkdir -p {DST_DIR}/imagesTr {DST_DIR}/labelsTr {DST_DIR}/imagesTs {DST_DIR}/imagesVal {DST_DIR}/labelsVal

In [ ]:
#Train
for subtype in ["subtype0", "subtype1", "subtype2"]:
    src_path = f"{SRC_DIR}/train/{subtype}"
    imgs = glob.glob(f"{src_path}/*_0000.nii.gz")
    for img in imgs:
        mask = img.replace("_0000.nii.gz", ".nii.gz")
        shutil.copy(img, f"{DST_DIR}/imagesTr/")
        shutil.copy(mask, f"{DST_DIR}/labelsTr/")

#Validate
for subtype in ["subtype0", "subtype1", "subtype2"]:
    src_path = f"{SRC_DIR}/validation/{subtype}"
    imgs = glob.glob(f"{src_path}/*_0000.nii.gz")
    for img in imgs:
        mask = img.replace("_0000.nii.gz", ".nii.gz")
        shutil.copy(img, f"{DST_DIR}/imagesVal/")
        shutil.copy(mask, f"{DST_DIR}/labelsVal/")

#Test
for img in glob.glob(f"{SRC_DIR}/test/*.nii.gz"):
    shutil.copy(img, f"{DST_DIR}/imagesTs/")

In [ ]:
csv_path = f"{DST_DIR}/subtype_labels.csv"
with open(csv_path, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["Filename", "Subtype"])
    for subtype in ["subtype0", "subtype1", "subtype2"]:
        for split in ["train"]:
            imgs = glob.glob(f"{SRC_DIR}/{split}/{subtype}/*_0000.nii.gz")
            for img in imgs:
                base = os.path.basename(img).replace("_0000.nii.gz", "")
                writer.writerow([base, subtype[-1]])

In [8]:
num_training = len([f for f in os.listdir(f"{DST_DIR}/imagesTr") if f.endswith(".nii.gz")])
num_test = len([f for f in os.listdir(f"{DST_DIR}/imagesTs") if f.endswith(".nii.gz")])

dataset_json = {
    "channel_names": {
        "0": "CT"
    },
    "labels": {
        "background": 0,
        "pancreas": 1,
        "lesion": 2
    },
    "numTraining": num_training,
    "file_ending": ".nii.gz",
    "converted_by": "Ryan Khalloqi"
}

json_path = f"{DST_DIR}/dataset.json"
with open(json_path, "w") as f:
    json.dump(dataset_json, f, indent=4)

print(json.dumps(dataset_json, indent=4))

{
    "channel_names": {
        "0": "CT"
    },
    "labels": {
        "background": 0,
        "pancreas": 1,
        "lesion": 2
    },
    "numTraining": 252,
    "file_ending": ".nii.gz",
    "converted_by": "Ryan Khalloqi"
}


Dataset Check

In [ ]:
#round pixel vals

label_dirs = [
    f"{DST_DIR}/labelsTr",
    f"{DST_DIR}/labelsVal"
]

for label_dir in label_dirs:
    for path in glob.glob(f"{label_dir}/*.nii.gz"):
        img = nib.load(path)
        data = img.get_fdata()

        data = np.rint(data).astype(np.uint8)
        data[data > 2] = 2
        data[data < 0] = 0

        nib.save(nib.Nifti1Image(data, img.affine, img.header), path)

In [9]:
!nnUNetv2_plan_and_preprocess -d 310 -pl nnUNetPlannerResEncM --verify_dataset_integrity

Fingerprint extraction...
Dataset310_Pancreas
Using <class 'nnunetv2.imageio.simpleitk_reader_writer.SimpleITKIO'> as reader/writer

####################
verify_dataset_integrity Done. 
If you didn't see any error messages then your dataset is most likely OK!
####################

Using <class 'nnunetv2.imageio.simpleitk_reader_writer.SimpleITKIO'> as reader/writer
100% 252/252 [00:16<00:00, 14.94it/s]
Experiment planning...
Dropping 3d_lowres config because the image size difference to 3d_fullres is too small. 3d_fullres: [ 59. 118. 181.], 3d_lowres: [59, 118, 181]
2D U-Net configuration:
{'data_identifier': 'nnUNetPlans_2d', 'preprocessor_name': 'DefaultPreprocessor', 'batch_size': 134, 'patch_size': (np.int64(128), np.int64(192)), 'median_image_size_in_voxels': array([118., 181.]), 'spacing': array([0.73046875, 0.73046875]), 'normalization_schemes': ['CTNormalization'], 'use_mask_for_norm': [False], 'resampling_fn_data': 'resample_data_or_seg_to_shape', 'resampling_fn_seg': 'resampl

...

In [12]:
!cp /content/drive/MyDrive/PC-Classifier/model/multitask_trainer.py /usr/local/lib/python3.12/dist-packages/nnunetv2/training/nnUNetTrainer/

In [14]:
!cp /content/drive/MyDrive/PC-Classifier/model/classification_head.py /usr/local/lib/python3.12/dist-packages/nnunetv2/training/nnUNetTrainer/

In [16]:
!echo "" > /usr/local/lib/python3.12/dist-packages/nnunetv2/training/nnUNetTrainer/__init__.py

In [17]:
!ls -l /usr/local/lib/python3.12/dist-packages/nnunetv2/training/nnUNetTrainer/

total 96
-rw-------  1 root root   780 Oct 28 04:14 classification_head.py
-rw-r--r--  1 root root     1 Oct 28 04:18 __init__.py
-rw-------  1 root root  2344 Oct 28 04:17 multitask_trainer.py
-rw-r--r--  1 root root 71846 Oct 28 03:46 nnUNetTrainer.py
drwxr-xr-x  3 root root  4096 Oct 28 03:46 primus
drwxr-xr-x  2 root root  4096 Oct 28 04:15 __pycache__
drwxr-xr-x 12 root root  4096 Oct 28 03:46 variants


In [19]:
import sys
sys.path.append("/usr/local/lib/python3.12/dist-packages")

from nnunetv2.training.nnUNetTrainer.multitask_trainer import MultiTaskTrainer
from nnunetv2.training.nnUNetTrainer.classification_head import ClassificationHead

In [24]:
import inspect
from nnunetv2.training.nnUNetTrainer.nnUNetTrainer import nnUNetTrainer

print(inspect.signature(nnUNetTrainer.__init__))

(self, plans: dict, configuration: str, fold: int, dataset_json: dict, device: torch.device = device(type='cuda'))


In [30]:
!nnUNetv2_train 310 3d_fullres 0 \
  -p nnUNetResEncUNetMPlans \
  -tr MultiTaskTrainer

Using device: cuda:0

#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################

MultiTaskTrainer initialized (network not yet built)
2025-10-28 04:39:45.651490: Using torch.compile...
wandb: Currently logged in as: ryanwangk (ryanwangk-university-of-wisconsin-madison) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.
wandb: ⢿ Waiting for wandb.init()...
wandb: ⣻ Waiting for wandb.init()...
wandb: Tracking run with wandb version 0.22.2
wandb: Run data is saved locally in /content/wandb/run-20251028_043947-wt5wnvd2


In [32]:
!ls -lh /content/drive/MyDrive/PC-Classifier/nnUNet_results/Dataset310_Pancreas/MultiTaskTrainer__nnUNetResEncUNetMPlans__3d_fullres/fold_0/

total 1.6G
-rw------- 1 root root 781M Oct 28 05:29 checkpoint_best.pth
-rw------- 1 root root 781M Oct 28 05:25 checkpoint_latest.pth
-rw------- 1 root root  13K Oct 28 04:39 debug.json
-rw------- 1 root root 541K Oct 28 05:35 progress.png
-rw------- 1 root root  404 Oct 28 04:30 training_log_2025_10_28_04_30_04.txt
-rw------- 1 root root 3.3K Oct 28 04:31 training_log_2025_10_28_04_31_10.txt
-rw------- 1 root root  456 Oct 28 04:37 training_log_2025_10_28_04_37_44.txt
-rw------- 1 root root  456 Oct 28 04:38 training_log_2025_10_28_04_38_57.txt
-rw------- 1 root root  30K Oct 28 05:34 training_log_2025_10_28_04_39_42.txt


In [53]:
!nnUNetv2_predict \
  -i /content/drive/MyDrive/PC-Classifier/nnUNet_raw/Dataset310_Pancreas/imagesVal \
  -o /content/drive/MyDrive/PC-Classifier/predictions_val \
  -d 310 -c 3d_fullres \
  -p nnUNetResEncUNetMPlans \
  -tr MultiTaskTrainer \
  -f 0 \
  -chk checkpoint_best.pth


#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################

There are 36 cases in the source folder
I am processing 0 out of 1 (max process ID is 0, we start counting with 0!)
There are 36 cases that I would like to predict

Predicting quiz_0_168:
perform_everything_on_device: True
100% 4/4 [00:01<00:00,  2.41it/s]
sending off prediction to background worker for resampling and export
done with quiz_0_168

Predicting quiz_0_171:
perform_everything_on_device: True
100% 2/2 [00:00<00:00,  4.66it/s]
sending off prediction to background worker for resampling and export
done with quiz_0_171

Predicting quiz_0_174:
perform_everything_on_device: True
100% 3

In [56]:
pred_dir = "/content/drive/MyDrive/PC-Classifier/predictions_val"

base_dir = "/content/drive/MyDrive/PC-Classifier"
pred_dir = f"{base_dir}/predictions_val"
input_dir = f"{base_dir}/nnUNet_raw/Dataset310_Pancreas/imagesVal"
label_dir = f"{base_dir}/nnUNet_raw/Dataset310_Pancreas/labelsVal"

case = sorted([f for f in os.listdir(pred_dir) if f.endswith(".nii.gz")])[0]
case_id = os.path.splitext(os.path.splitext(case)[0])[0]
print("Visualizing:", case_id)

pred_img = nib.load(os.path.join(pred_dir, case))
pred_data = pred_img.get_fdata()

img_img = nib.load(os.path.join(input_dir, f"{case_id}_0000.nii.gz"))
img_data = img_img.get_fdata()

gt_img = nib.load(os.path.join(label_dir, f"{case_id}.nii.gz"))
gt_data = gt_img.get_fdata()

mid = img_data.shape[2] // 2

overlay = np.zeros((*pred_data.shape, 3))
overlay[pred_data == 1] = [1, 0, 0]
overlay[pred_data == 2] = [0, 1, 0]

plt.figure(figsize=(18, 6))
plt.subplot(1, 3, 1)
plt.imshow(img_data[:, :, mid], cmap="gray")
plt.title("Original CT")
plt.axis("off")

plt.subplot(1, 3, 2)
plt.imshow(img_data[:, :, mid], cmap="gray")
plt.imshow(overlay[:, :, mid], alpha=0.5)
plt.title("Segmentation Prediction")
plt.axis("off")

plt.subplot(1, 3, 3)
plt.imshow(img_data[:, :, mid], cmap="gray")
plt.imshow(gt_data[:, :, mid], cmap="jet", alpha=0.4)
plt.title("Ground Truth")
plt.axis("off")

plt.tight_layout()
plt.show()

Visualizing: quiz_0_168


In [57]:
plt.savefig("/content/drive/MyDrive/PC-Classifier/sample_visualization.png", dpi=200)

In [58]:
def dice_score(pred, target, label):
    pred_bin = (pred == label).astype(np.uint8)
    target_bin = (target == label).astype(np.uint8)
    intersection = np.sum(pred_bin * target_bin)
    denom = np.sum(pred_bin) + np.sum(target_bin)
    return (2. * intersection / denom) if denom > 0 else np.nan

whole_pred = (pred_data > 0).astype(np.uint8)
whole_gt = (gt_data > 0).astype(np.uint8)
dsc_whole = dice_score(whole_pred, whole_gt, 1)

dsc_lesion = dice_score(pred_data, gt_data, 2)

print(f"Whole pancreas DSC: {dsc_whole:.3f}")
print(f"Pancreas lesion DSC: {dsc_lesion:.3f}")

Whole pancreas DSC: 0.874
Pancreas lesion DSC: 0.012


In [62]:
import pandas as pd

base_dir = "/content/drive/MyDrive/PC-Classifier"
pred_dir = f"{base_dir}/predictions_val"
gt_dir = f"{base_dir}/nnUNet_raw/Dataset310_Pancreas/labelsVal"

def dice_score(pred, gt):
    intersection = np.sum((pred > 0) & (gt > 0))
    return (2.0 * intersection) / (np.sum(pred > 0) + np.sum(gt > 0) + 1e-8)

results = []
cases = sorted([f for f in os.listdir(pred_dir) if f.endswith(".nii.gz")])

for case in cases:
    case_id = os.path.splitext(os.path.splitext(case)[0])[0]
    pred_file = os.path.join(pred_dir, case)
    gt_file = os.path.join(gt_dir, f"{case_id}.nii.gz")

    if not os.path.exists(gt_file):
        print("⚠️ Missing ground truth for:", case_id)
        continue

    pred = nib.load(pred_file).get_fdata()
    gt = nib.load(gt_file).get_fdata()

    whole_dsc = dice_score(pred > 0, gt > 0)
    lesion_dsc = dice_score(pred == 2, gt == 2)
    results.append([case_id, whole_dsc, lesion_dsc])

df = pd.DataFrame(results, columns=["Case", "Whole_Pancreas_DSC", "Lesion_DSC"])

print(df)
print("\nAverage Results:\n", df[["Whole_Pancreas_DSC", "Lesion_DSC"]].mean())

          Case  Whole_Pancreas_DSC  Lesion_DSC
0   quiz_0_168            0.874281    0.011908
1   quiz_0_171            0.886987    0.737191
2   quiz_0_174            0.829151    0.650201
3   quiz_0_184            0.932848    0.916279
4   quiz_0_187            0.926402    0.340285
5   quiz_0_189            0.923552    0.756631
6   quiz_0_244            0.936325    0.421370
7   quiz_0_253            0.920367    0.486115
8   quiz_0_254            0.902114    0.225460
9   quiz_1_090            0.848980    0.060250
10  quiz_1_093            0.866815    0.776573
11  quiz_1_094            0.935983    0.310248
12  quiz_1_154            0.889943    0.000000
13  quiz_1_158            0.908620    0.803649
14  quiz_1_164            0.899609    0.895549
15  quiz_1_166            0.932364    0.839276
16  quiz_1_211            0.874290    0.628875
17  quiz_1_213            0.903426    0.841381
18  quiz_1_221            0.953211    0.794614
19  quiz_1_227            0.914680    0.000000
20  quiz_1_23

          Case  Whole_Pancreas_DSC  Lesion_DSC
0   quiz_0_168            0.874281    0.011908
1   quiz_0_171            0.886987    0.737191
2   quiz_0_174            0.829151    0.650201
3   quiz_0_184            0.932848    0.916279
4   quiz_0_187            0.926402    0.340285
5   quiz_0_189            0.923552    0.756631
6   quiz_0_244            0.936325    0.421370
7   quiz_0_253            0.920367    0.486115
8   quiz_0_254            0.902114    0.225460
9   quiz_1_090            0.848980    0.060250
10  quiz_1_093            0.866815    0.776573
11  quiz_1_094            0.935983    0.310248
12  quiz_1_154            0.889943    0.000000
13  quiz_1_158            0.908620    0.803649
14  quiz_1_164            0.899609    0.895549
15  quiz_1_166            0.932364    0.839276
16  quiz_1_211            0.874290    0.628875
17  quiz_1_213            0.903426    0.841381
18  quiz_1_221            0.953211    0.794614
19  quiz_1_227            0.914680    0.000000
20  quiz_1_231            0.945854    0.594331
21  quiz_1_242            0.933307    0.000000
22  quiz_1_331            0.893210    0.023695
23  quiz_1_335            0.920233    0.821231
24  quiz_2_074            0.948543    0.870815
25  quiz_2_080            0.936716    0.697407
26  quiz_2_084            0.923531    0.850914
27  quiz_2_085            0.929293    0.800543
28  quiz_2_088            0.774661    0.000000
29  quiz_2_089            0.882018    0.885896
30  quiz_2_098            0.907120    0.859250
31  quiz_2_191            0.747997    0.000000
32  quiz_2_241            0.958289    0.612441
33  quiz_2_364            0.878694    0.633799
34  quiz_2_377            0.873241    0.826760
35  quiz_2_379            0.953031    0.802966

Average Results:
 Whole_Pancreas_DSC    0.901825
Lesion_DSC            0.549331
dtype: float64

In [61]:
import json
split_file = "/content/drive/MyDrive/PC-Classifier/nnUNet_preprocessed/Dataset310_Pancreas/splits_final.json"

with open(split_file) as f:
    splits = json.load(f)

print("Fold 0 has", len(splits[0]['val']), "validation cases")

Fold 0 has 51 validation cases
